# 2D Turbulence: From Equations to GPU Implementation

In this lecture, we will build a 2D incompressible Navier-Stokes solver from scratch in Julia. Along the way, we'll learn:

1. **The governing equations** of incompressible fluid flow
2. **Finite volume discretization** on a staggered grid
3. **Extensible programming** using Julia's multiple dispatch
4. **Advection schemes** and how to extend them
5. **FFT-based pressure solvers** for periodic domains

GPU computing basics and KernelAbstractions.jl are covered in the [GPU Computing notebook](01_gpu_computing.ipynb).

By the end, you'll have a working turbulence simulation that runs on both CPU and GPU!
This notebook follows quite closely the implementation of Oceananigans.jl, so you'll know what happens behind the 
scenes when you run a simulation!

---
## 1. The Governing Equations

We solve the **2D incompressible Navier-Stokes equations**:

$$\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla)\mathbf{u} = -\nabla p + \nu \nabla^2 \mathbf{u}$$

$$\nabla \cdot \mathbf{u} = 0$$

Where:
- $\mathbf{u} = (u, v)$ is the velocity field
- $p$ is the pressure (divided by density)
- $\nu$ is the kinematic viscosity

### Physical Interpretation

| Term | Name | Physical Meaning |
|------|------|------------------|
| $\partial \mathbf{u}/\partial t$ | Time derivative | Rate of change of velocity |
| $(\mathbf{u} \cdot \nabla)\mathbf{u}$ | Advection | Fluid carries momentum with it |
| $-\nabla p$ | Pressure gradient | Fluid flows from high to low pressure |
| $\nu \nabla^2 \mathbf{u}$ | Diffusion | Viscosity smooths out velocity gradients |
| $\nabla \cdot \mathbf{u} = 0$ | Incompressibility | Fluid volume is conserved |

### The Projection Method

We use the **projection method** (Chorin, 1968) to solve these equations. The idea is to split each time step into a *predictor* that ignores the pressure, and a *correction* that enforces incompressibility.

**Step 1 — Predictor (ignore pressure):**

Advance the velocity using only advection and diffusion:

$$\mathbf{u}^* = \mathbf{u}^n + \Delta t \left[ -(\mathbf{u} \cdot \nabla)\mathbf{u} + \nu \nabla^2 \mathbf{u} \right]$$

The predicted velocity $\mathbf{u}^*$ is generally **not divergence-free** ($\nabla \cdot \mathbf{u}^* \neq 0$).

**Step 2 — Correction (enforce incompressibility):**

We want to find a pressure $p$ such that the corrected velocity is divergence-free:

$$\mathbf{u}^{n+1} = \mathbf{u}^* - \Delta t \, \nabla p$$

**Step 3 — Deriving the Poisson equation:**

We require $\nabla \cdot \mathbf{u}^{n+1} = 0$. Taking the divergence of the correction:

$$\nabla \cdot \mathbf{u}^{n+1} = \nabla \cdot \mathbf{u}^* - \Delta t \, \nabla^2 p = 0$$

Rearranging:

$$\boxed{\nabla^2 p = \frac{\nabla \cdot \mathbf{u}^*}{\Delta t}}$$

This is a **Poisson equation** for $p$, with the divergence of the predicted velocity as the source term. Once solved, the pressure gradient projects $\mathbf{u}^*$ onto the space of divergence-free fields.

**Summary of the algorithm:**

1. **Compute tendencies**: $G = -(\mathbf{u} \cdot \nabla)\mathbf{u} + \nu \nabla^2 \mathbf{u}$
2. **Predictor step**: $\mathbf{u}^* = \mathbf{u}^n + \Delta t \cdot G$
3. **Pressure solve**: $\nabla^2 p = \frac{\nabla \cdot \mathbf{u}^*}{\Delta t}$
4. **Correction**: $\mathbf{u}^{n+1} = \mathbf{u}^* - \Delta t \, \nabla p$

The key insight: the pressure acts as a **Lagrange multiplier** that enforces incompressibility. The name "projection" comes from the fact that the correction step is an orthogonal projection of $\mathbf{u}^*$ onto the subspace of divergence-free vector fields.

---
## 2. The Staggered Grid (Arakawa C-grid)

We use a **staggered grid** where different variables live at different locations:

```
    +-------v[i,j+1]-------+
    |                      |
  u[i,j]     p[i,j]     u[i+1,j]
    |                      |
    +--------v[i,j]--------+
```

- **Pressure** $p$: cell centers
- **u-velocity**: west/east faces (between pressure points in x)
- **v-velocity**: south/north faces (between pressure points in y)

### Why Staggered?

1. **No pressure checkerboard**: Collocated grids can develop spurious pressure oscillations
2. **Exact discrete incompressibility**: $\nabla \cdot \mathbf{u} = 0$ is satisfied to machine precision

---
## 3. Setup

We covered GPU computing and KernelAbstractions.jl in the [GPU Computing notebook](01_gpu_computing.ipynb). Here we load the packages we need:

In [ ]:
using CUDA
using KernelAbstractions
using LinearAlgebra

---
## 4. Extensible Programming with Multiple Dispatch

Julia's **multiple dispatch** allows us to write extensible code. Instead of:

```python
# Python-style (single dispatch)
if scheme == "second_order":
    result = second_order_reconstruct(f, i)
elif scheme == "fourth_order":
    result = fourth_order_reconstruct(f, i)
```

We write:

```julia
# Julia-style (multiple dispatch)
reconstruct(f, i, ::SecondOrder) = ...  # Method for SecondOrder
reconstruct(f, i, ::FourthOrder) = ...  # Method for FourthOrder
```

### Benefits:
1. **Adding new schemes doesn't touch existing code** - just add new methods
2. **Type-stable** - the compiler knows exactly which method to call
3. **Composable** - schemes can be combined without special cases

In [ ]:
##### Example: Extensible face reconstruction schemes
# Given values at cell centers, reconstruct the value at face i+1/2

abstract type AbstractScheme end
struct SecondOrderCentered <: AbstractScheme end
struct FourthOrderCentered <: AbstractScheme end

# 2nd order: average of two neighbors
reconstruct(f, i, ::SecondOrderCentered) = (f[i] + f[i+1]) / 2

# 4th order: uses a wider stencil for higher accuracy
reconstruct(f, i, ::FourthOrderCentered) = (-f[i-1] + 7f[i] + 7f[i+1] - f[i+2]) / 12

# Test: f(x) = x² at cell centers x = 0, 1, 2, 3, 4
# Reconstruct at face between cells 3 and 4, i.e. at x = 2.5
f = [0.0, 1.0, 4.0, 9.0, 16.0]  # f = x²
exact = 2.5^2  # = 12.25

f2 = reconstruct(f, 3, SecondOrderCentered())
f4 = reconstruct(f, 3, FourthOrderCentered())

println("2nd order reconstruction: ", f2, "  (error = ", f2 - exact, ")")
println("4th order reconstruction: ", f4, "  (error = ", f4 - exact, ")")
println("Exact value at x = 2.5:  ", exact)

---
## 5. Building the Solver: Grid Structure

Now let's build our solver piece by piece. First, the grid:

In [ ]:
# Architecture aliases for cleaner code
const CPU = KernelAbstractions.CPU
const GPU = CUDA.CUDABackend

# Functions to get array types for each architecture
array_type(::CPU) = Array
array_type(::GPU) = CuArray

In [ ]:
struct Grid{T, A, AR}
    architecture :: AR   # CPU() or GPU()
    Nx :: Int            # Number of cells in x
    Ny :: Int            # Number of cells in y
    Lx :: T              # Domain length in x
    Ly :: T              # Domain length in y
    Δx :: T              # Cell width
    Δy :: T              # Cell height
    x  :: A              # Cell center x-coordinates
    y  :: A              # Cell center y-coordinates
end

function Grid(arch, Nx, Ny, Lx, Ly)
    T  = typeof(Lx)
    Δx = Lx / Nx
    Δy = Ly / Ny
    AT = array_type(arch)
    x  = AT(T[(i - 0.5) * Δx for i in 1:Nx])
    y  = AT(T[(j - 0.5) * Δy for j in 1:Ny])
    return Grid(arch, Nx, Ny, Lx, Ly, Δx, Δy, x, y)
end

Base.size(grid::Grid)    = (grid.Nx, grid.Ny)
architecture(grid::Grid) = grid.architecture
array_type(grid::Grid)   = array_type(architecture(grid))

In [ ]:
# Test it
grid_cpu = Grid(CPU(), 64, 64, 2π, 2π)
println("Grid size: ", size(grid_cpu))
println("Cell size: Δx = ", grid_cpu.Δx, ", Δy = ", grid_cpu.Δy)

---
## 6. Advection Schemes

The advection term $(\mathbf{u} \cdot \nabla)\mathbf{u}$ describes how the fluid carries momentum.

### Finite Volume Approach

We compute the **flux** of momentum through each cell face:
$$\frac{\partial uu}{\partial x} + \frac{\partial uv}{\partial y} \ \ \ \text{in the u equation}$$
$$\frac{\partial uv}{\partial x} + \frac{\partial vv}{\partial y} \ \ \ \text{in the v equation}$$
For example, for cell $(i,j)$, the advection of $u$ is:
$$-\frac{F^u_{i+1/2} - F^u_{i-1/2}}{\Delta x} - \frac{G^u_{j+1/2} - G^u_{j-1/2}}{\Delta y}$$
Where $F^u = uu$ and $G^u = uv$ are the fluxes of momentum in the $x$ and $y$ directions.

### The Key Question: how do we compute these fluxes?

To compute the flux $F = U \cdot u$ at a face, we need:
1. The **advecting velocity** $U$ at the face
2. The **advected quantity** $u$ at the face

But we only know values at cell centers! We must **reconstruct** face values.

### Flux Computation on the C-grid: What Gets Interpolated Where?

On a staggered C-grid, velocities and fluxes live at different locations.
To compute a flux like $F = U \cdot \phi$ at a face, we often need to **reconstruct** or **interpolate** quantities to locations where they aren't naturally defined.

Let's visualize all four advective flux combinations for the momentum equations.

#### Advection of $u$ by $U$ (same direction — x-fluxes of u-momentum)

$u$ lives on **x-faces** (●). To compute the flux $F^{uu}$ at the **cell center** between two $u$-points, we **reconstruct** both $U$ and $u$ there:

```
         u-faces:     ● ------⊠------ ● -------⊠------- ●
                    u[i-1]          u[i]              u[i+1]

         flux at:             ⊠                ⊠
                            i-1/2            i+1/2

    U at flux point:  avg(u[i-1], u[i])   avg(u[i], u[i+1])    ← interpolate U
    u at flux point:  reconstruct(u, i-1)  reconstruct(u, i)    ← reconstruct u
```

Both $U$ and $u$ need reconstruction to the **midpoints** between $u$-points.

---

#### Advection of $u$ by $V$ (cross direction — y-fluxes of u-momentum)

$u$ lives on **x-faces** (●), $v$ lives on **y-faces** (▲). The y-flux of $u$ lives at the **corner** of the cell:

```
                  |                |
                  |                |
                  ▲ v[i-1,j+1]     ▲ v[i,j+1]
                  |                |
                  |                |
          ● ------⊠------ ● -------⊠------- ●
        u[i-1,j+1]|    u[i,j+1]    |    u[i+1,j+1]
                  |                |
                  ▲ v[i-1,j]       ▲ v[i,j]
                  |                |
                  |                |
          ● ------⊠------ ● -------⊠------- ●
        u[i-1,j]  |     u[i,j]     |     u[i+1,j]
                  |                |
                  ▲ v[i-1,j-1]     ▲ v[i,j-1]
                  |                |
                  |                |


    ⊠ = flux location (corner point)
```

At each ⊠ we need:
- **$V$**: interpolate $v$ in $x$ → average two neighboring $v$-points
- **$u$**: reconstruct $u$ in $y$ → reconstruct from $u$-values above and below

---

**Key pattern**: The advecting velocity is always **averaged** (interpolated) to the flux location, while the advected quantity is **reconstructed** using the chosen advection scheme.
This is where the dispatch on the advection scheme will come in!

### Second-Order Centered Scheme

The simplest approach: average neighboring values:

$$u_{i+1/2} = \frac{u_i + u_{i+1}}{2}$$

This is **second-order accurate** but can be unstable for advection-dominated flows.
Let's start writing this algorithm!

In [ ]:
# Index helpers for periodic boundary conditions
@inline  left(i, N) = ifelse(i == 1, N, i - 1)
@inline right(i, N) = ifelse(i == N, 1, i + 1)

In [ ]:
# Advection scheme types
abstract type AbstractAdvectionScheme end
struct Centered <: AbstractAdvectionScheme end

In [ ]:
# Symmetric (centered) interpolation to the RIGHT face (i+1/2)
@inline symmetric_x(i, j, f, N) = @inbounds (f[i, j] + f[right(i, N), j]) / 2
@inline symmetric_y(i, j, f, N) = @inbounds (f[i, j] + f[i, right(j, N)]) / 2

In [ ]:
# Face reconstruction dispatched by scheme
# For Centered scheme: just use symmetric interpolation
@inline reconstruct_in_x(i, j, f, N, U, ::Centered) = symmetric_x(i, j, f, N)
@inline reconstruct_in_y(i, j, f, N, V, ::Centered) = symmetric_y(i, j, f, N)

### Computing Advective Fluxes

Now we build the flux functions. We need four combinations:

| Function | Meaning | Grid Relationship |
|----------|---------|-------------------|
| `advective_flux_Uu` | U advects u | Same grid (x-direction) |
| `advective_flux_Vu` | V advects u | Cross grid (y-direction) |
| `advective_flux_Uv` | U advects v | Cross grid (x-direction) |
| `advective_flux_Vv` | V advects v | Same grid (y-direction) |

In [ ]:
# U advecting u (same grid, x-direction)
# Returns (flux_right, flux_left)
@inline function advective_flux_Uu(i, j, u, Nx, s)
    im = left(i, Nx)
    Uᴿ = symmetric_x(i,  j, u, Nx)
    Uᴸ = symmetric_x(im, j, u, Nx)
    return Uᴿ * reconstruct_in_x(i,  j, u, Nx, Uᴿ, s),
           Uᴸ * reconstruct_in_x(im, j, u, Nx, Uᴸ, s)
end

# V advecting v (same grid, y-direction)
@inline function advective_flux_Vv(i, j, v, Ny, s)
    jm = left(j, Ny)
    Vᴿ = symmetric_y(i, j,  v, Ny)
    Vᴸ = symmetric_y(i, jm, v, Ny)
    return Vᴿ * reconstruct_in_y(i, j,  v, Ny, Vᴿ, s),
           Vᴸ * reconstruct_in_y(i, jm, v, Ny, Vᴸ, s)
end

# V advecting u (cross grid - V needs interpolation to u's location)
@inline function advective_flux_Vu(i, j, u, v, Nx, Ny, s)
    im = left(i, Nx)
    jm = left(j, Ny)
    Vᴿ = symmetric_x(im, right(j, Ny), v, Nx)   # V at north face of u-cell (y_{j+1/2})
    Vᴸ = symmetric_x(im, j,            v, Nx)   # V at south face of u-cell (y_{j-1/2})
    return Vᴿ * reconstruct_in_y(i, j,  u, Ny, Vᴿ, s),
           Vᴸ * reconstruct_in_y(i, jm, u, Ny, Vᴸ, s)
end

# U advecting v (cross grid - U needs interpolation to v's location)
@inline function advective_flux_Uv(i, j, u, v, Nx, Ny, s)
    im = left(i, Nx)
    jm = left(j, Ny)
    Uᴿ = symmetric_y(right(i, Nx), jm, u, Ny)   # U at east face of v-cell (x_{i+1/2})
    Uᴸ = symmetric_y(i,            jm, u, Ny)   # U at west face of v-cell (x_{i-1/2})
    return Uᴿ * reconstruct_in_x(i,  j, v, Nx, Uᴿ, s),
           Uᴸ * reconstruct_in_x(im, j, v, Nx, Uᴸ, s)
end

---
## 7. Pressure Solver

The pressure Poisson equation for a doubly-periodic domain:

$$\nabla^2 p = f$$

can be solved efficiently using the **Fast Fourier Transform (FFT)**.

### Why FFT Works

The Laplacian $\nabla^2$ is diagonal in Fourier space:

$$\nabla^2 p = f \quad \Rightarrow \quad -(k_x^2 + k_y^2) \hat{p} = \hat{f}$$

So the algorithm is:
1. FFT the right-hand side: $\hat{f} = \mathcal{F}(f)$
2. Divide by eigenvalues: $\hat{p} = -\hat{f} / (k_x^2 + k_y^2)$
3. Inverse FFT: $p = \mathcal{F}^{-1}(\hat{p})$

**Complexity**: $O(N^2 \log N)$ vs $O(N^4)$ for direct methods!

### Why Discrete Eigenvalues, Not $k_x^2 + k_y^2$?

You might expect to use the **continuous** Laplacian eigenvalues $k_x^2 + k_y^2$ where $k_x = 2\pi m / L_x$. But we must use the **discrete** Laplacian eigenvalues instead!

**The reason**: We're solving the *discrete* Poisson equation, not the continuous one. Our discrete Laplacian is:

$$(\nabla^2 p)_{i,j} = \frac{p_{i+1,j} - 2p_{i,j} + p_{i-1,j}}{\Delta x^2} + \frac{p_{i,j+1} - 2p_{i,j} + p_{i,j-1}}{\Delta y^2}$$

When we substitute $p_{i,j} = \hat{p} \cdot e^{i(k_x x_i + k_y y_j)}$ into this stencil, we get:

$$\nabla^2_{\text{discrete}} \rightarrow -\frac{4\sin^2(\pi m / N_x)}{\Delta x^2} - \frac{4\sin^2(\pi n / N_y)}{\Delta y^2}$$

**Not** $-k_x^2 - k_y^2$!

### What happens if we use the wrong eigenvalues?

If we used $k_x^2 + k_y^2$:
- The pressure solve would give the wrong answer
- The correction step $\mathbf{u}^{n+1} = \mathbf{u}^* - \Delta t \nabla p$ wouldn't properly remove divergence
- We'd lose the beautiful property that $\nabla \cdot \mathbf{u} = 0$ to machine precision

**Key insight**: The FFT pressure solver must be *consistent* with the discrete Laplacian used elsewhere in the code. We're inverting the discrete operator, not the continuous one!

In [ ]:
using FFTW
using AbstractFFTs  # For GPU-compatible FFT planning

In [ ]:
struct FFTPoissonSolver{E, S, P, IP}
    eigenvalues :: E  # Laplacian eigenvalues in Fourier space
    storage     :: S  # Workspace for FFT
    plan        :: P  # Forward FFT plan
    iplan       :: IP # Inverse FFT plan
end

function FFTPoissonSolver(grid::Grid{T}) where T
    Nx, Ny = grid.Nx, grid.Ny
    Δx, Δy = grid.Δx, grid.Δy
    AT = array_type(grid)

    # Discrete Laplacian eigenvalues: λ = 4sin²(πm/Nx)/Δx² + 4sin²(πn/Ny)/Δy²
    # Built on CPU first, then transferred (avoids scalar indexing on GPU)
    eigenvalues = T[
        4 * sin(π * (i-1) / Nx)^2 / Δx^2 + 4 * sin(π * ifelse(j <= Ny÷2+1, j-1, j-1-Ny) / Ny)^2 / Δy^2
        for i in 1:Nx÷2+1, j in 1:Ny
    ]
    eigenvalues[1, 1] = T(Inf)    # DC mode: 1/Inf = 0 (pressure defined up to constant)
    eigenvalues = AT(eigenvalues) # Move to GPU if needed

    storage  = AT(zeros(Complex{T}, Nx÷2+1, Ny))
    rhs_temp = AT(zeros(T, Nx, Ny))
    plan     = AbstractFFTs.plan_rfft(rhs_temp)
    iplan    = AbstractFFTs.plan_irfft(storage, Nx)

    return FFTPoissonSolver(eigenvalues, storage, plan, iplan)
end

function solve!(p, rhs, solver::FFTPoissonSolver)
    LinearAlgebra.mul!(solver.storage, solver.plan, rhs)     # FFT
    solver.storage .= .-solver.storage ./ solver.eigenvalues # Solve in Fourier space
    LinearAlgebra.mul!(p, solver.iplan, solver.storage)      # Inverse FFT
    return nothing
end

---
## 8. Finite Difference Operators

We need a few more operators for the diffusion term and pressure correction:

In [ ]:
# Forward differences
@inline δx⁺(i, j, f, Δx, Nx) = @inbounds (f[right(i, Nx), j] - f[i, j]) / Δx
@inline δy⁺(i, j, f, Δy, Ny) = @inbounds (f[i, right(j, Ny)] - f[i, j]) / Δy

# Backward differences  
@inline δx⁻(i, j, f, Δx, Nx) = @inbounds (f[i, j] - f[left(i, Nx), j]) / Δx
@inline δy⁻(i, j, f, Δy, Ny) = @inbounds (f[i, j] - f[i, left(j, Ny)]) / Δy

# Laplacian: ∇²f = ∂²f/∂x² + ∂²f/∂y²
@inline function ∇²(i, j, f, Δx, Δy, Nx, Ny)
    @inbounds begin
        ∂²x = (f[right(i, Nx), j] - 2*f[i, j] + f[left(i, Nx), j]) / Δx^2
        ∂²y = (f[i, right(j, Ny)] - 2*f[i, j] + f[i, left(j, Ny)]) / Δy^2
        return ∂²x + ∂²y
    end
end

---
## 9. GPU Kernels

Now we write the kernels that will run on both CPU and GPU:

In [ ]:
# Compute momentum tendencies (advection + diffusion)
@kernel function compute_tendencies!(Gu, Gv, u, v, ν, Δx, Δy, Nx, Ny, scheme)
    i, j = @index(Global, NTuple)

    @inbounds begin
        # u-momentum: -∂(Uu)/∂x - ∂(Vu)/∂y + ν∇²u
        Fux⁺, Fux⁻ = advective_flux_Uu(i, j, u, Nx, scheme)
        Fuy⁺, Fuy⁻ = advective_flux_Vu(i, j, u, v, Nx, Ny, scheme)
        Gu[i, j] = -(Fux⁺ - Fux⁻) / Δx - (Fuy⁺ - Fuy⁻) / Δy + ν * ∇²(i, j, u, Δx, Δy, Nx, Ny)

        # v-momentum: -∂(Uv)/∂x - ∂(Vv)/∂y + ν∇²v
        Fvx⁺, Fvx⁻ = advective_flux_Uv(i, j, u, v, Nx, Ny, scheme)
        Fvy⁺, Fvy⁻ = advective_flux_Vv(i, j, v, Ny, scheme)
        Gv[i, j] = -(Fvx⁺ - Fvx⁻) / Δx - (Fvy⁺ - Fvy⁻) / Δy + ν * ∇²(i, j, v, Δx, Δy, Nx, Ny)
    end
end

# Predictor step: u* = uⁿ + Δt × G
@kernel function predict!(u, v, Gu, Gv, Δt)
    i, j = @index(Global, NTuple)
    @inbounds begin
        u[i, j] = u[i, j] + Δt * Gu[i, j]
        v[i, j] = v[i, j] + Δt * Gv[i, j]
    end
end

# Compute divergence for pressure solve
@kernel function compute_divergence!(div, u, v, Δx, Δy, Nx, Ny)
    i, j = @index(Global, NTuple)
    @inbounds div[i, j] = δx⁺(i, j, u, Δx, Nx) + δy⁺(i, j, v, Δy, Ny)
end

# Correction step: uⁿ⁺¹ = u* - Δt∇p
@kernel function correct!(u, v, p, Δt, Δx, Δy, Nx, Ny)
    i, j = @index(Global, NTuple)
    @inbounds begin
        u[i, j] -= Δt * δx⁻(i, j, p, Δx, Nx)
        v[i, j] -= Δt * δy⁻(i, j, p, Δy, Ny)
    end
end

---
## 10. The Solver Structure

Now we put everything together:

In [ ]:
struct NavierStokesModel{T, G, A, S, ADV}
    grid      :: G
    u         :: A    # u-velocity
    v         :: A    # v-velocity
    p         :: A    # pressure
    Gu        :: A    # u-tendency
    Gv        :: A    # v-tendency
    div       :: A    # divergence
    ω         :: A    # vorticity (for diagnostics)
    ν         :: T    # viscosity
    advection :: ADV  # advection scheme
    poisson_solver :: S
end

function NavierStokesModel(grid::Grid{T}, ν; advection=Centered()) where T
    Nx, Ny = size(grid)
    AT = array_type(grid)

    u   = AT(zeros(T, Nx, Ny))
    v   = AT(zeros(T, Nx, Ny))
    p   = AT(zeros(T, Nx, Ny))
    Gu  = AT(zeros(T, Nx, Ny))
    Gv  = AT(zeros(T, Nx, Ny))
    div = AT(zeros(T, Nx, Ny))
    ω   = AT(zeros(T, Nx, Ny))

    poisson_solver = FFTPoissonSolver(grid)

    return NavierStokesModel(grid, u, v, p, Gu, Gv, div, ω, T(ν), advection, poisson_solver)
end

In [ ]:
function time_step!(model::NavierStokesModel, Δt)
    (; grid, u, v, p, Gu, Gv, div, ν, advection, poisson_solver) = model
    (; Nx, Ny, Δx, Δy) = grid
    
    arch = architecture(grid)
    worksize  = (Nx, Ny)
    workgroup = (16, 16)

    # 1. Compute tendencies (advection + diffusion)
    compute_tendencies!(arch, workgroup)(Gu, Gv, u, v, ν, Δx, Δy, Nx, Ny, advection, ndrange=worksize)

    # 2. Predictor step
    predict!(arch, workgroup)(u, v, Gu, Gv, Δt, ndrange=worksize)

    # 3. Compute divergence of predicted velocity
    compute_divergence!(arch, workgroup)(div, u, v, Δx, Δy, Nx, Ny, ndrange=worksize)
    div ./= Δt

    # 4. Solve for pressure
    solve!(p, div, poisson_solver)

    # 5. Correction step
    correct!(arch, workgroup)(u, v, p, Δt, Δx, Δy, Nx, Ny, ndrange=worksize)

    return nothing
end

---
## 11. Initial Conditions: Random Vortices

We initialize with random **Lamb-Oseen vortices** ([Lamb, 1932](https://archive.org/details/hydrodynamics00LambH); [Oseen, 1912](https://doi.org/10.1007/BF02418652)). The vorticity of a Lamb-Oseen vortex is a Gaussian:

$$\omega = \frac{\Gamma}{\pi \sigma^2} e^{-r^2/\sigma^2}$$

where $\Gamma$ is the circulation and $\sigma$ is the core size.

### From Vorticity to Velocity via Streamfunction

Rather than computing velocity directly from the analytical formula (which involves long-range $1/r$ interactions that break periodicity), we use the **streamfunction** $\psi$:

1. **Set vorticity**: Sum of Gaussian vortices (compact, naturally periodic)
2. **Solve for streamfunction**: $\nabla^2 \psi = -\omega$ using our FFT solver
3. **Compute velocity**: $u = \partial\psi/\partial y$, $v = -\partial\psi/\partial x$

This reuses the FFT pressure solver and guarantees a smooth, periodic velocity field with no boundary artifacts!

In [ ]:
using Random

# Kernel to set vorticity as a sum of Gaussian vortices
@kernel function initialize_vorticity_kernel!(ω, x_vort, y_vort, Γ_vort, σ, Δx, Δy, Lx, Ly, n_vortices)
    i, j = @index(Global, NTuple)
    xc = (i - 0.5) * Δx
    yc = (j - 0.5) * Δy

    ω_val = zero(eltype(ω))
    for n in 1:n_vortices
        dx = xc - x_vort[n]
        dy = yc - y_vort[n]
        # Periodic distance (minimum image convention)
        dx = dx - Lx * round(dx / Lx)
        dy = dy - Ly * round(dy / Ly)
        r² = dx^2 + dy^2
        ω_val += Γ_vort[n] / (π * σ^2) * exp(-r² / σ^2)
    end
    @inbounds ω[i, j] = ω_val
end

# Kernel to compute velocity from streamfunction: u = ∂ψ/∂y, v = -∂ψ/∂x
@kernel function velocity_from_streamfunction!(u, v, ψ, Δx, Δy, Nx, Ny)
    i, j = @index(Global, NTuple)
    @inbounds begin
        u[i, j] =  δy⁻(i, j, ψ, Δy, Ny)
        v[i, j] = -δx⁻(i, j, ψ, Δx, Nx)
    end
end

function initialize_random_vortices!(model; n_vortices=20, σ=0.1, Γ_max=1.0, seed=1234)
    (; grid, u, v, ω, p, poisson_solver) = model
    (; Nx, Ny, Lx, Ly, Δx, Δy) = grid
    AT = array_type(grid)
    T  = eltype(Lx)

    Random.seed!(seed)
    x_vort = AT(rand(T, n_vortices) .* Lx)
    y_vort = AT(rand(T, n_vortices) .* Ly)
    Γ_vort = AT((2 .* rand(T, n_vortices) .- 1) .* Γ_max)

    arch = architecture(grid)
    worksize  = (Nx, Ny)
    workgroup = (16, 16)

    # 1. Initialize vorticity as sum of Gaussian vortices
    initialize_vorticity_kernel!(arch, workgroup)(ω, x_vort, y_vort, Γ_vort, σ, Δx, Δy, Lx, Ly, n_vortices, ndrange=worksize)

    # 2. Solve for streamfunction: ∇²ψ = -ω (reuse pressure array for ψ)
    ω .*= -1
    solve!(p, ω, poisson_solver)  # p now holds ψ
    ω .*= -1  # restore ω

    # 3. Compute velocity from streamfunction
    velocity_from_streamfunction!(arch, workgroup)(u, v, p, Δx, Δy, Nx, Ny, ndrange=worksize)

    return nothing
end

## 12. Utilities for running a simulation

Let's add a method for easily running a simulation, for visualizing some diagnostic and to generate a nice video.
For visualization we use the Makie package.


In [ ]:
# Compute vorticity for visualization
@kernel function compute_vorticity!(ω, u, v, Δx, Δy, Nx, Ny)
    i, j = @index(Global, NTuple)
    @inbounds ω[i, j] = δx⁺(i, j, v, Δx, Nx) - δy⁺(i, j, u, Δy, Ny)
end

# Helper to compute vorticity for visualization
function compute_vorticity!(model)
    (; grid, u, v, ω) = model
    (; Nx, Ny, Δx, Δy) = grid
    arch = architecture(grid)
    compute_vorticity!(arch, (16, 16))(ω, u, v, Δx, Δy, Nx, Ny, ndrange=(Nx, Ny))
    return nothing
end

# Helper to compute the divergence for diagnostic purposes
function compute_divergence!(model::NavierStokesModel)
    (; grid, u, v, div) = model
    (; Δx, Δy, Nx, Ny) = grid
    arch = architecture(grid)
    compute_divergence!(arch, (16, 16))(div, u, v, Δx, Δy, Nx, Ny, ndrange=(Nx, Ny))
    return nothing
end
    
function run!(model::NavierStokesModel, Δt, Tfinal; save_every=500)
    n_steps = ceil(Int, Tfinal / Δt)
    
    # Storage for animation frames
    compute_vorticity!(model)
    ωhist = [Array(model.ω) |> copy]
    uhist = [Array(model.u) |> copy]
    vhist = [Array(model.v) |> copy]
    phist = [Array(model.p) |> copy] 
    thist = [0.0]
    
    println("Running $n_steps steps (saving every $save_every)...")
    @time for n in 1:n_steps
        time_step!(model, Δt)
        
        if n % save_every == 0
            compute_vorticity!(model)
            push!(ωhist, Array(model.ω) |> copy)
            push!(uhist, Array(model.u) |> copy)
            push!(vhist, Array(model.v) |> copy)
            push!(phist, Array(model.p) |> copy)
            push!(thist, n * Δt)
        end

        if n % 5000 == 0
            compute_divergence!(model)
            div_max = maximum(abs.(Array(model.div)))
            @info "Step $n/$n_steps: t=$(round(n*Δt, digits=3)), |∇·u|=$(round(div_max, sigdigits=3))"
        end
    end

    return ωhist, uhist, vhist, phist, thist
end

In [ ]:
using CairoMakie

function plot_history(history, filename)
    x_plot = Array(grid.x)
    y_plot = Array(grid.y)
    ω_lim = maximum(abs.(history[1][1])) * 0.8
    u_lim = maximum(abs.(history[2][1])) * 0.8
    v_lim = maximum(abs.(history[3][1])) * 0.8
    p_lim = maximum(abs.(history[4][end])) * 0.8
    
    n  = Observable(1)
    ωn = @lift(history[1][$n])
    un = @lift(history[2][$n])
    vn = @lift(history[3][$n])
    pn = @lift(history[4][$n])
    tn = @lift(string(round(history[5][$n], digits=2)))
    
    fig = Figure(size=(800, 800))
    ax1 = Axis(fig[1, 1], title=@lift("u at " * $tn), xlabel="x", ylabel="y", aspect=DataAspect())
    ax2 = Axis(fig[1, 2], title=@lift("v at " * $tn), xlabel="x", ylabel="y", aspect=DataAspect())
    ax3 = Axis(fig[2, 1], title=@lift("ω at " * $tn), xlabel="x", ylabel="y", aspect=DataAspect())
    ax4 = Axis(fig[2, 2], title=@lift("p at " * $tn), xlabel="x", ylabel="y", aspect=DataAspect())
    
    hm = heatmap!(ax1, x_plot, y_plot, un, colormap=:RdBu, colorrange=(-u_lim, u_lim))
    hm = heatmap!(ax2, x_plot, y_plot, vn, colormap=:RdBu, colorrange=(-v_lim, v_lim))
    hm = heatmap!(ax3, x_plot, y_plot, ωn, colormap=:RdBu, colorrange=(-ω_lim, ω_lim))
    hm = heatmap!(ax4, x_plot, y_plot, pn, colormap=:RdBu, colorrange=(-p_lim, p_lim))
    
    CairoMakie.record(fig, filename * ".mp4", 1:length(history[1])) do i
        n[] = i
    end
end

And one for embedding a video in the jupyter notebook!

In [ ]:
using Base64
function embed_mp4(path, width=900)
    data = base64encode(open(path))
    HTML("""<video controls autoplay loop width="$width"><source src="data:video/mp4;base64,$data" type="video/mp4"></video>""")
end

---
## 12. Running on CPU and Creating a Video

Let's run a full simulation and create a video of the evolving vorticity field!

In [ ]:
# Create model on CPU
Nx, Ny = 256, 256
Lx, Ly = 2π, 2π
ν = 5e-4 # 1 / Re

grid  = Grid(CPU(), Nx, Ny, Lx, Ly)
model = NavierStokesModel(grid, ν; advection=Centered())

# Initialize with random vortices
initialize_random_vortices!(model; n_vortices=30, σ=0.15, Γ_max=2.0)

# Run the simulation!
hist = run!(model, 0.0005, 10.0; save_every=100)
nothing # hide

In [ ]:
# Visualize...
plot_history(hist, "cpu_turbulence")
embed_mp4("cpu_turbulence.mp4")

---
## 13. High-Resolution GPU Simulation

Now let's unleash the GPU! We'll run a **1024×1024** simulation - that's **16× more grid points** than our CPU run.

This demonstrates the real power of GPUs:
- More grid points = more parallelism = better GPU utilization
- The same code runs without modification
- We can resolve finer turbulent structures

In [ ]:
if CUDA.functional()
    # High-resolution GPU simulation!
    Nx, Ny = 1024, 1024 
    grid  = Grid(GPU(), Nx, Ny, Lx, Ly)
    ν = 5e-5
    model = NavierStokesModel(grid, ν; advection=Centered())
    initialize_random_vortices!(model; n_vortices=50, σ=0.1, Γ_max=2.0)  # More vortices!
    hist = run!(model, 0.00002, 10.0; save_every=1000)
    plot_history(hist, "gpu_turbulence")
    embed_mp4("gpu_turbulence.mp4")
else
    println("CUDA not available - skipping GPU example")
end

---
## 14. Exercise: Implement Upwind Advection

The centered scheme we used is simple but can be **numerically unstable** for advection-dominated flows. Upwind schemes are more stable because they respect the direction of information flow.

### Third-Order Upwind Scheme

Instead of symmetric interpolation, we bias the stencil based on flow direction.

For reconstruction at face $i+1/2$:

**If $U > 0$ (flow going right), use left-biased stencil:**
$$f_{i+1/2} = \frac{-f_{i-1} + 5f_i + 2f_{i+1}}{6}$$

**If $U < 0$ (flow going left), use right-biased stencil:**
$$f_{i+1/2} = \frac{2f_i + 5f_{i+1} - f_{i+2}}{6}$$

### Your Task

1. Define a new type `struct UpwindThirdOrder <: AbstractAdvectionScheme end`
2. Implement the biased stencils `left_biased_x`, `left_biased_y`, `right_biased_x`, `right_biased_y`
3. Implement `reconstruct_in_x` and `reconstruct_in_y` methods for `UpwindThirdOrder`

**Hints**: 
- Use `ifelse(U >= 0, left_biased, right_biased)` to select the appropriate stencil.
- Remember to use the `left` and `right` methods to access i-1 and i+1 correctly. 
- i+2 can be accessed combining multiple calls to `right`: `right(right(i, N), N)`

In [ ]:
# Exercise: Implement UpwindThirdOrder

# Step 1: Define the type
struct UpwindThirdOrder <: AbstractAdvectionScheme end

# Step 2: Implement biased stencils (to RIGHT face i+1/2)
# Left-biased: (-f[i-1] + 5f[i] + 2f[i+1]) / 6
@inline left_biased_x(i, j, f, N) = # ... YOUR CODE HERE 
@inline left_biased_y(i, j, f, N) = # ... YOUR CODE HERE

# Right-biased: (2f[i] + 5f[i+1] - f[i+2]) / 6
@inline right_biased_x(i, j, f, N) = # ... YOUR CODE HERE
@inline right_biased_y(i, j, f, N) = # ... YOUR CODE HERE

# Step 3: Implement reconstruction methods
# Use ifelse to select stencil based on velocity sign
@inline reconstruct_in_x(i, j, f, N, U, ::UpwindThirdOrder) = ifelse(U ≥ 0, # ... YOUR CODE HERE
@inline reconstruct_in_y(i, j, f, N, V, ::UpwindThirdOrder) = ifelse(V ≥ 0, # ... YOUR CODE HERE

In [ ]:
# Test your implementation!
# Uncomment when ready:
# Nx, Ny = 256, 256
# grid  = Grid(CPU(), Nx, Ny, Lx, Ly) # you can also use GPU() here for faster results!!
# ν = 1e-4
# model = NavierStokesModel(grid, ν; advection=UpwindThirdOrder())
# initialize_random_vortices!(model; n_vortices=30, σ=0.15, Γ_max=2.0)
# hist = run!(model, 0.0001, 10.0)
# plot_history(hist, "upwind_turbulence")
# embed_mp4("upwind_turbulence.mp4")

### Bonus Challenge: Fifth-Order Upwind

For the brave! The fifth-order upwind scheme uses a 5-point stencil:

**Left-biased ($U > 0$):**
$$f_{i+1/2} = \frac{2f_{i-2} - 13f_{i-1} + 47f_i + 27f_{i+1} - 3f_{i+2}}{60}$$

**Right-biased ($U < 0$):**
$$f_{i+1/2} = \frac{-3f_{i-1} + 27f_i + 47f_{i+1} - 13f_{i+2} + 2f_{i+3}}{60}$$

Can you implement `UpwindFifthOrder`?

In [ ]:
# Bonus: Implement UpwindFifthOrder
# struct UpwindFifthOrder <: AbstractAdvectionScheme end
# ...

---
### Key Takeaways

- **Julia + KernelAbstractions** = write once, run on CPU/GPU
- **Multiple dispatch** makes code extensible without modification
- **FFT** is the key to fast pressure solves in periodic domains
- **Staggered grids** avoid pressure checkerboarding

### Further Reading

- [Chorin's Projection Method (1968)](https://doi.org/10.1090/S0025-5718-1968-0242392-2)
- [KernelAbstractions.jl Documentation](https://juliagpu.github.io/KernelAbstractions.jl/stable/)
- [CUDA.jl Documentation](https://cuda.juliagpu.org/stable/)
- [Oceananigans.jl](https://github.com/CliMA/Oceananigans.jl) - A production CFD code using these techniques